# News search using kNN in Hopsworks

In this tutorial, you are going to learn how to create a news search application which allows you to search news using natural language. You will create embedding for the news and search news similar to a given description using embeddings and kNN search. You will also learn how to avoid training-serving skew by using model registry. The steps include:
1. Load news data
2. Create embedddings for news heading and news body
3. Save the embedding model to model registry
4. Ingest the news data and embedding into Hopsworks
5. Search news using Hopsworks

## Load news data

First, you need to load the news articles downloaded from [Kaggle news articles](https://www.kaggle.com/datasets/asad1m9a9h6mood/news-articles).
Since creating embeddings for the full news is time-consuming, here we sample some articles.

In [ ]:
import pandas as pd

df_all = pd.read_csv("https://repo.hops.works/dev/jdowling/Articles.csv", encoding='utf-8', encoding_errors='ignore')
df = df_all.sample(n=300).reset_index().drop(["index"], axis=1)
df["news_id"] = list(range(len(df)))

## Create embeddings

Next, you need to create embeddings for heading and body of the news. The embeddings will then be used for kNN search against the embedding of the news description you want to search. Here we use a light weighted language model (LM) which encodes the news into embeddings. You can use any other language models including LLM (llama, Mistral).

In [ ]:
!pip install sentence_transformers -q

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# truncate the body to 100 characters
embeddings_body = model.encode([body for body in df["Article"]])
embeddings_heading = model.encode(df["Heading"])
df["embedding_heading"] = pd.Series(embeddings_heading.tolist())
df["embedding_body"] = pd.Series(embeddings_body.tolist())

In [ ]:
df.head()

## Ingest into Hopsworks

You need to ingest the data to Hopsworks, so that they are stored and indexed. First, you login into Hopsworks and prepare the feature store.

In [ ]:
import hopsworks
proj = hopsworks.login()
fs = proj.get_feature_store()

Next, as embeddings are stored in an index in the backing vecotor database, you need to specify the index name and the embedding features in the dataframe. You can also save the embedding model to model registry, and attach the model to the embedding features. This is useful for avoiding training-serving skew as at inference time you can get back the same model used for creating embedding at training time.

First, you save the model to model registry.

In [ ]:
import pickle
model_name = "SentenceTransformer_all_MiniLM_L6_v2"
mr = proj.get_model_registry()
# Check if the model has been created, and get back the model if available. Otherwise create the model.
try:
    hsml_model = mr.get_model(model_name, 1)
except:
    with open(f"{model_name}.pkl", "wb") as f:
        pickle.dump(model, f)
    hsml_model = mr.python.create_model(model_name)
    hsml_model.save(f"{model_name}.pkl")


Then, you specify the index name, embedding features, and model.

In [ ]:
version = 1
from hsfs import embedding

emb = embedding.EmbeddingIndex(index_name=f"news_fg_{version}")
# specify the name, dimension, and model of the embedding features 
emb.add_embedding("embedding_body", model.get_sentence_embedding_dimension(), model=hsml_model)
emb.add_embedding("embedding_heading", model.get_sentence_embedding_dimension(), model=hsml_model)

Next, you create a feature group with the `embedding_index` and ingest data into the feature group.

In [ ]:
news_fg = fs.get_or_create_feature_group(
    name="news_fg",
    embedding_index=emb,
    primary_key=["news_id"],
    version=version,
    online_enabled=True
)

news_fg.insert(df, write_options={"start_offline_materialization": False})

## Search News

Once the data are ingested into Hopsworks, you can search news by giving a news description. The news description first needs to be encoded by the same LM you used to encode the news. You can get back the model in the model registry from the embedding feature. And then you can search news which are similar to the description using kNN search functionality provided by the feature group.

In [ ]:
# set the logging level to WARN to avoid INFO message
import logging
logging.getLogger().setLevel(logging.WARN)

In [ ]:
news_description = "news about europe"

You can get back the model file from embedding feature object, and load the model file back to the embedding model.

In [ ]:
hsml_model = news_fg.embedding_index.get_embedding("embedding_heading").model
local_model_path = hsml_model.download()
with open(f"{local_model_path}/{hsml_model.name}.pkl", 'rb') as f:
    loaded_model = pickle.load(f)

You can search similar news to the description against news heading.

In [ ]:
results = news_fg.find_neighbors(loaded_model.encode(news_description), k=3, col="embedding_heading")
# print out the heading
for result in results:
    print(result[1][2])

Alternative, you can search similar news to the description against the news body and filter by news type.

In [ ]:
results = news_fg.find_neighbors(loaded_model.encode(news_description), k=3, col="embedding_body",
                                filter=news_fg.newstype == "business")
# print out the heading
for result in results:
    print(result[1][2])

## Next step

Now you are able to search articles using natural language. You can learn how to rank the result in [this tutorial]() and learn best practices in the [guide]().